<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Adding-a-field-to-meshed-bodies-display-files" data-toc-modified-id="Adding-a-field-to-meshed-bodies-display-files-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Adding a field to meshed bodies display files</a></span><ul class="toc-item"><li><span><a href="#Preparation" data-toc-modified-id="Preparation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Preparation</a></span><ul class="toc-item"><li><span><a href="#Simple-sample-generation" data-toc-modified-id="Simple-sample-generation-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Simple sample generation</a></span></li></ul></li><li><span><a href="#Computation" data-toc-modified-id="Computation-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Computation</a></span><ul class="toc-item"><li><span><a href="#Simulation-parameters-definition" data-toc-modified-id="Simulation-parameters-definition-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simulation parameters definition</a></span></li><li><span><a href="#Computation-with-usual-visualization" data-toc-modified-id="Computation-with-usual-visualization-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Computation with usual visualization</a></span></li><li><span><a href="#Adding-a-field-to-Paraview" data-toc-modified-id="Adding-a-field-to-Paraview-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Adding a field to Paraview</a></span></li></ul></li></ul></li></ul></div>

# Adding a field to meshed bodies display files


It is possible within a __pylmgc90.chipy__ python script, to add new fields to the display files generated by the `chipy.WriteDisplayFiles` function.

But depending on the file to write, the syntax may be a little cumbersome. Here will be presented a
simple case showing how to do this for the `mecafe` block of the `lmgc90` paraview files describing the mechanical bodies modeled with finite elements method.  
Here the `mecafe`block is used, but the syntax is exactly the same for `therfe`, `porofe` and `multife` blocks.

In the following we prepare a new sample, build a computation script and improve it to add new fields to display files. 


## Preparation

### Simple sample generation

Let's consider a simple plate made of an elastic material. And in order to illustrate an external
parameter, let us take into account thermal dilatency. And to make things a little more difficult,
two different material parameters (for Young's modulus and thermal dilatency) will be used on different
part of the plate.

This is done by using two user defined materials with the MatLib and defining a group.


In [ ]:
from pathlib import Path
import shutil, math

import numpy as np

from pylmgc90 import pre

datbox = Path('./DATBOX')
datbox.mkdir(exist_ok=True)

# 2D
dim = 2

# containers
bodies = pre.avatars()
mats   = pre.materials()
mods   = pre.models()
posts  = pre.postpro_commands()

# user model in hpp
mod = pre.model(name='Q4MLx', physics='MECAx', element='Q4xxx', dimension=2,
                user_model_name='ISOTROPIC_THERMO_DILATANT_ELASTICITY',
                external_model='MatL_', kinematic='small', mass_storage='coher',
                external_fields=['TEMPERATURE'])
mods.addModel(mod)

# materials definition in file
steel1 = pre.material(name='stee1', materialType='USER_MAT', density=0., file_mat='elas2.mat')
mats.addMaterial(steel1)

steel2 = pre.material(name='stee2', materialType='USER_MAT', density=0., file_mat='elas1.mat')
mats.addMaterial(steel2)

# beam mesh generation
beam_mesh = pre.buildMesh2D(mesh_type='Q4', x0=0., y0=0., lx=0.0032, ly=0.0324, nb_elem_x=6, nb_elem_y=36)

# avatar building
beam = pre.buildMeshedAvatar(mesh=beam_mesh, model=mod, material=steel1)

# add a group to split the plate in two different parts
# along a slanted line
def line(x):
    return x[1] < 5.0625*x[0]+0.81e-2

beam.addGroupUsingPredicate(name='lower_part', predicate=line)

# redefining of the material on created group
beam.defineMaterial(group='lower_part',material=steel2)
beam.checkModelAndMaterialDefinitions()

# Boundary conditions
beam.imposeDrivenDof(group='up'  , component=2, dofty='vlocy')
beam.imposeDrivenDof(group='down', component=2, dofty='vlocy')

def p(x):
    return abs(x[0] - 0.0016) < 5.e-4

beam.addGroupUsingPredicate(name='mooring', predicate=p, super_group='down')
beam.imposeDrivenDof(group='mooring', component=1, dofty='vlocy')

bodies.addAvatar(beam)

# Input files writing
pre.writeDatbox(dim, mats, mods, bodies, post=posts, gravy=[0.,0.,0.], datbox_path=datbox)

#pre.visuAvatars(bodies)
shutil.copy('elas1.mat','DATBOX')
shutil.copy('elas2.mat','DATBOX')
# for f in ['elas1.mat', 'elas2.mat']:
#     src = Path(f)
#     dst = datbox/f
#     dst.write_text(src.read_text())

The user material is defined in file containing the following informations:

```shell
!elastic model

! elastic properties----------------------------------
YOUNG_MODULUS                  =  0.1170e12 (real)
POISSON_COEFFICIENT            =  0.350   (real)

! thermoelastic properties

THERMAL_DILATATION_COEFFICIENT =  1.0e-5 (real)
REFERENCE_TEMPERATURE          =  20.   (real)
INITIAL_TEMPERATURE            =  20.   (real)
```

These data are saved in the file `elas1.mat`. A second file `elas2.mat`
is created with the same structure but different values for Young moduleu
and thermal dilatation coefficient. These two files must be put in the `DATBOX` directory.

## Computation

Compared to previous examples, the example used here needs to:
1. call the `chipy.mecaMAILx_SetScalarFieldByNode` to set the external field value in the mesh before each *computation* step,
2. deactivate the default visualization step of the *computation* step.

### Simulation parameters definition

Definition of all the parameters to compute the simulation:

In [ ]:
from pylmgc90 import chipy
from pylmgc90.chipy import computation

import numpy as np

# space dimension
dim = 2

# modeling hypothesis ( 1 = plain strain, 2 = plain stress, 3 = axi-symmetry)
mhyp = 1

# time evolution parameters
dt = 1.e-0
nb_steps = 50

# theta integrator parameter
theta = 1.

# nlgs parameters define in a dictionnary to earn some space
stype = 'Stored_Delassus_Loops         '
norm  = 'Quad '
conv  = 1e-4
relax = 1.0
gsit1 = 50
gsit2 = 10

# write parameter
freq_write   = 10

# display parameters
freq_display = 10


### Computation with usual visualization

So, now that the parameters are all defined, an usual computation
is done like this:

In [ ]:
computation.initialize( dim, dt, theta, mhyp, 'lmgc90.h5', deformable=True )

T0 = 20.
T = T0 * np.ones( chipy.mecaMAILx_GetNbNodes(1) )
chipy.mecaMAILx_SetScalarFieldByNode(1,1,T)

for k in range(1,nb_steps+1):

    if k%10 == 0:
        print( f"computing step {k}" )

    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
    chipy.WriteDisplayFiles(freq_display)

    # uniform temperature increase
    T[:] += 10.
    # you are more than welcome to try any changes you would like
    #T[:5] += 10
    #T[:-5] -= 10

    # Setting value of the first field of first mecaMAILx body
    chipy.mecaMAILx_SetScalarFieldByNode(1,1,T)


computation.finalize()
print('Computation finished')

You can check the result in paraview (if you run paraview from here,
remember to close it before trying to run new cells).

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview

### Adding a field to Paraview

On deformable bodies, the field values can be attached to the nodes of the
mesh or to the elements. 

In this exercice, it is desired to add fields to paraview to visualize:
* the temperature at nodes.
* the material id on the elements.

Furthermore, even if there is only one mesh in this example, there may be several bodies.
Paraview needs to define the desired fields for all bodies involved in a single `mecafe` block.

So a list of arrays must be provided (one numpy array holding field values for each body).

The temperature will be updated at each time step, but the material will be defined only once before the time loop, even it is added to the visualization at each time step.

In [ ]:
# Initialization (reading database and so on)
computation.initialize( dim, dt, theta, mhyp, 'lmgc90.h5', deformable=True )

T0 = 20.
T = T0 * np.ones( chipy.mecaMAILx_GetNbNodes(1) )
chipy.mecaMAILx_SetScalarFieldByNode(1,1,T)

# list containing the temperature nodes array of the body
display_T = [T]
# list containing the material id of elements of body
display_mat = [ chipy.mecaMAILx_GetMaterials(1) ]

Now, during the time loop modify the call to `chipy.WriteDisplayFile` to
add the new field to display:

In [ ]:
for k in range(1,nb_steps+1):

    if k%10 == 0:
        print( f"computing step {k}" )

    computation.one_step(stype, norm, conv, relax, gsit1, gsit2, freq_write, 0)
    # add the list to the visualization, 'T_ext' is my choice
    # and is the name of the field added in paraview
    chipy.WriteDisplayFiles(freq_display, T_ext=('mecafe', 'node', display_T),
                                          material=('mecafe', 'element', display_mat)
                           )

    # uniform temperature increase
    T[:] += 10.
    # you are more than welcome to try any changes you would like
    #T[:5] += 10
    #T[:-5] -= 10

    # Setting value of the first field of first mecaMAILx body
    chipy.mecaMAILx_SetScalarFieldByNode(1,1,T)


computation.finalize()
print('Computation finished')

Now check in paraview that the `T_ext` and `material` (or any other name you choose if you changed them) fields
appear in the *mecafe* display files.

In [ ]:
#running a specific version of paraview on macos
#!/Applications/ParaView-5.3.0-RC2.app/Contents/MacOS/paraview

#running a specific version of paraview on Linux
#!paraview